In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import json
import time

In [4]:
base_url = "https://www.racingchannel.com"
text = requests.get(base_url+"/results_archive.php").text
soup = BeautifulSoup(text, "html.parser")
table = soup.find("div", attrs={"class": "AccordionPanelContent"}).find_all("a")

In [5]:
locations = [url.get_text() for url in table] # get location text for each anchored element in table
# clean typos in original table
locations[1] = unicode("Arlington Park")
locations[34] = unicode("Indiana Downs")
loc_urls = [url.get("href") for url in table] # get url for each anchored element in table

In [6]:
%%time
date_dict = {}
for loc, loc_url in zip(locations, loc_urls):
    r1 = requests.get(base_url+loc_url).text
    s1 = BeautifulSoup(r1, "html.parser")
    cols = s1.find_all("td", attrs={"valign": "top"})
    links = []
    date_format = []
    dates = []
    for col in cols:
        months = col.find_all("a")
        for month in months:
            links.append(month.get("href"))
    for link in links:
        r2 = requests.get(base_url+link).text
        s2 = BeautifulSoup(r2, "html.parser")
        cal_urls = s2.find("table", attrs={"width": "567"}).find_all("a")
        for cal_url in cal_urls:
            dates.append(cal_url.get('href'))
        time.sleep(2)
    import re # http://regexr.com
    for item in dates:
        l = re.findall("(?:\d*\.)?\d+", item)
        date_format.append(l[0] + l[1] + l[2][4:])
    date_dict[loc] = date_format

CPU times: user 3min 39s, sys: 11.8 s, total: 3min 51s
Wall time: 4h 23min 23s


In [129]:
#date_dict

In [128]:
#locations

In [6]:
zips = ['11420', '60005', 'NA', 'NA', 'NA', '94403', '11003', '43123', '33056', '00729', '55379', '25438',
        '40208', '23124', '70668', '92014', '19804', '42420', '98001', '70570', '70119', '62234', '91768', 
        '14425', 'NA', '94710', 'NA', 'NA', '49415', '33009', 'NA', '60804', '90305', '46013', '46176', '40510', 
        '42134', '20725', '71111', '75050', '90720', '07073', '07757', '26047', 'NA', '71901', '17028', 
        '19020', '21215', '97217', '50009', '73111', '78154', '45230', '77064', '91007', '12866', 'NA', 
        'NA', '60804', '02128', '33626', '44128', '21093', '85023', '41042', 'NA'] 

In [7]:
zips_dict = dict(zip(locations, zips))

In [126]:
#zips_dict

In [68]:
date = '20141225'
zip_code = '11420'
wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(date, zip_code)

In [69]:
examp = json.loads(requests.get(wunderground_url).text)

In [201]:
weather_data = ['fog','hail','maxhumidity','meandewptm','meanpressurem','meantempm','meanvism',
                'meanwdird', 'meanwindspdm', 'precipm', 'rain', 'snow', 'snowdepthm','snowfallm', 'thunder',
                'minhumidity']

In [183]:
def output_dict(in_dict):
    temp = [(elem,in_dict[elem]) for elem in in_dict.keys() if elem in weather_data]
    return dict(temp)

In [202]:
output_dict(examp['history']['dailysummary'][0])

{u'fog': u'1',
 u'hail': u'0',
 u'maxhumidity': u'96',
 u'meandewptm': u'3',
 u'meanpressurem': u'1008',
 u'meantempm': u'10',
 u'meanvism': u'13',
 u'meanwdird': u'261',
 u'meanwindspdm': u'27',
 u'minhumidity': u'35',
 u'precipm': u'1.52',
 u'rain': u'1',
 u'snow': u'0',
 u'snowdepthm': u'0.00',
 u'snowfallm': u'0.00',
 u'thunder': u'0'}

In [77]:
datesdf = resudf=pd.read_csv("tempdata/usadays.csv")
datesdf.drop('track.1',axis=1, inplace=True)
datesdf.head()

,track,year,month,day
0,AQU,1998,10,28
1,AQU,1998,10,29
2,AQU,1998,10,30
3,AQU,1998,10,31
4,AQU,1998,11,1


In [103]:
datesdf[['year', 'month', 'day']] = datesdf[['year', 'month', 'day']].astype(str)
for line in datesdf.index:
    if len(datesdf.loc[line]['month']) == 1:
        datesdf.loc[line]['month'] = '0' + datesdf.loc[line]['month']
    if len(datesdf.loc[line]['day']) == 1:
        datesdf.loc[line]['day'] = '0' + datesdf.loc[line]['day']
datesdf.head()

,track,year,month,day
0,AQU,1998,10,28
1,AQU,1998,10,29
2,AQU,1998,10,30
3,AQU,1998,10,31
4,AQU,1998,11,01


In [108]:
dates_dict = {}
for track in datesdf.track.unique():
    datestring = []
    for row in datesdf.index:
        if datesdf.iloc[row]['track'] == track:
            datestring.append(str(datesdf.iloc[row]['year']) + str(datesdf.iloc[row]['month']) + str(datesdf.iloc[row]['day']))
    dates_dict[track] = datestring 

In [160]:
zips1 = ['11420', '60005','94403', '11003', '43123', '33056', '55379','40208', '23124', '25438', '70668', 
         '92014', '19804', '42420', '98001', '70570', '70119', '14425', '91768', '62234', '49415', '94710', 
         '33009', '60804', '46013', '90305', '46176', '40510', '42134', '20725', '71111', '75050', '90720', 
         '07073', '07757', '26047', '71901', '85023', '17028', '19020', '21215', '50009', '73111', '78154', 
         '45230', '77064', '91007', '12866', '60804', '02128', '33626', '44128', '21093', '41042']
locs = datesdf.track.unique()
zips_dict1 = dict(zip(locs, zips1))

In [191]:
%%time
weather_dict = {}
for key in dates_dict.keys():
    for fdate in dates_dict[key]:
        wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(fdate, zips_dict1[key])
        temp = json.loads(requests.get(wunderground_url).text)['history']['dailysummary'][0]        
        weather_dict[(key, fdate)] = output_dict(temp)

CPU times: user 13.8 ms, sys: 3.21 ms, total: 17 ms
Wall time: 13 s


In [204]:
bigdf = []
for tup in weather_json.keys():
    stuff = [tup[0],tup[1]]
    for key in sorted(weather_json[tup].keys()):
        stuff.append(weather_json[tup][key])
    bigdf.append(stuff)
weatherdf = pd.DataFrame(bigdf, columns = ['track', 'date'] + sorted(weather_json[weather_json.keys()[0]].keys()))
weatherdf.head()

,track,date,fog,hail,humidity,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,EMD,19980814,0,0,,90,15,1017,21,16,226,5,54,0.00,0,0,,0.00,0
1,EMD,19980815,0,0,,100,13,1016,17,11,202,8,58,1.52,0,0,,0.00,0


In [ ]:
weatherdf.to_csv('tempdata/weather.csv')

In [56]:
url = "http://www.ncdc.noaa.gov/cdo-web/api/v2/data"
querystring = {"datasetid":"GHCND","locationid":"ZIP:71111","startdate":"2010-05-01","enddate":"2010-05-01"}
headers = {"token": "CGGFJuGumVuKGnDfDyYdkDmmeTcMzIOr","cache-control": "no-cache","postman-token": "2c03d9cf-3d0d-0582-faa8-a5b369dfa568"}
response = requests.request("GET", url, headers=headers, params=querystring)
json.loads(response.text)

{u'metadata': {u'resultset': {u'count': 2, u'limit': 25, u'offset': 1}},
 u'results': [{u'attributes': u',,N,',
   u'datatype': u'PRCP',
   u'date': u'2010-05-01T00:00:00',
   u'station': u'GHCND:US1LABS0001',
   u'value': 10},
  {u'attributes': u',,N,',
   u'datatype': u'PRCP',
   u'date': u'2010-05-01T00:00:00',
   u'station': u'GHCND:US1LABS0003',
   u'value': 13}]}